# chapter 9. chapter 9. Recommendation System

## Python Recommendation System Package - Surprise

### Surprise 패키지 소개 

콘텐츠 기반 필터링, 아이템 기반 필터링 그리조 잠재 요인 기반 협업 필터링은 최적화나 수행 속도 측면에서 좀 더 보완이 필요하다. <br>
**파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중 하나인 Surprise**는 사이킷런과 유사한 API와 프레임워크를 제공한다. <br>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 1.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp38-cp38-macosx_10_9_x86_64.whl size=1163527 sha256=b4ac5487661a3948e36f9a6f325d00560dc72824532fc4f565cf720589489640
  Stored in directory: /Users/1001l1000/Library/Caches/pip/wheels/af/db/86/2c18183a80ba05da35bf0fb7417aac5cddbd93bcb1b92fd3ea
Successfully built scikit-surprise


**Surprise의 주요 장점**

- 다양한 추천 알고리즘
- Surprise 핵심 API는 사이킷런의 핵심 API와 유사한 이름으로 작성이 되어있다. 

### Surprise를 이용한 추천 시스템 구축 

추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 행렬 분해를 통한 잠재 요인 협업 필터링을 수행한다. 

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

**Surprise에서 새로운 데이터 로딩은 Dataset Class를 이용해서만 가능**하다. <br>
**주요 데이터가 Row Level 형태로 되어 있는 포맷의 데이터만 처리**한다. 

In [3]:
data = Dataset.load_builtin('ml-100k')

# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값을 부여
trainset, testset = train_test_split(data, test_size = .25, random_state = 0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 Y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/1001l1000/.surprise_data/ml-100k


과거 버전 데이터 파일의 분리 문자는 (\t)이다. <br>
Surprise에 사용자-아이템 평점 데이터를 적용할 떄 주의해야 할 점은 Row Level의 사용자-아이템 평점 데이터를 그대로 적용해야 한다는 것이다. <br>
Surprise는 자체적으로 Row Level의 데이터를 Column Level의 데이터로 변경하므로 원본이 Row Level 사용자-아이템 평점 데이터를 데이터 세트로 적용해야 한다. <br>

**SVD 잠재 요인 협업 필터링 수행**

algo = SVD()와 같이 알고리즘 객체를 생성한 후 이 알고리즘 객체에 fit(학습 데이터 세트)를 수행해 학습 데이터 세트 기반으로 추천 알고리즘을 학습한다. 

In [4]:
algo = SVD(random_state = 0)
algo.fit(trainset)

**학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행**한다. <br>
Surprise에서 추천을 예측하는 메서드는 **test()** 와 **predict()** 이다. <br>
test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드이다. 즉, 입력된 데이터 세트에 대해 추천 데이터 세트를 만들어준다. <br>
predict()는 개별 사용자와 영화에 대한 추천 평점을 반환한다. 

In [5]:
predictions = algo.test(testset)
print('prediction type : ', type(predictions), ' size : ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type :  <class 'list'>  size :  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.180754247821916, details={'was_impossible': False})]

'was_impossible'이 True이면 예측 값을 생성할 수 없는 데이터라는 의미이다. <br>

In [7]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측한다. <br>

In [8]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 한다. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


**테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가**한다. <br>

accuracy 모듈은 RMSE, MSE 등의 방법으로 추천 시스템의 성능 평가 정보를 제공한다. 

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

### Surprise 주요 모듈 소개 

**Dataset**

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**ㅡ

### Surprise 추천 알고리즘 클래스 

### 베이스 라인 평점 

### 교차 검증과 하이퍼 파라미터 튜닝 

### Surprise를 이용한 개인화 영화 추천 시스템 구축